# Eager Execution Tutorial: Basics

This notebook introduces the basics of using TensorFlow's eager execution capabilities. It covers concepts such as:

* Importing required packages
* Enabling eager execution
* Creating and using TensorFlow Tensors and Variables
* Using TensorFlow interactively
* Using GPUs with eager execution enabled

This notebook does *not* cover modeling topics, such as gradients.

# Step 1: Import Eager

The key imports for eager execution are the following:

In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

# Step 2: Enable eager execution

All future TensorFlow calls will execute the
underlying TensorFlow ops immediately:

In [2]:
tfe.enable_eager_execution()

# Step 3: Interactively Use TensorFlow!

Now you can call TensorFlow functions and get results, immediately! No more `tf.Sessions`!

TensorFlow will automatically wrap native Python types for you with operator overloading for TensorFlow Tensors.

In [3]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))
print("")

x = tf.constant(2)
y = tf.constant(3)
print(x * y + 1)

# Most TensorFlow ops are directly usable with eager execution, giving
# results immediately.
print(tf.contrib.signal.hamming_window(x * y + 1))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)

tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(
[0.08000001 0.31000003 0.77000004 1.         0.77       0.30999985
 0.08000001], shape=(7,), dtype=float32)


Numpy arrays are supported, too:

In [4]:
import numpy as np

ones = np.ones([3, 3])

print("numpy 3x3 matrix of 1s:")
print(ones)
print("")

print("Multiplied by 42:")
print(tf.multiply(ones, 42))

numpy 3x3 matrix of 1s:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

Multiplied by 42:
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)


# Step 4: Define and Print TensorFlow Variables

To define TensorFlow variables, use the `get_variable()` function as follows:

In [5]:
x = tf.get_variable(name="x", shape=[], dtype=tf.float32, initializer=tf.zeros_initializer)

## Printing TensorFlow Variables

In [6]:
# This does NOT print the Variable's actual value:
print("Printing a TensorFlow Variable:")
print(x)
print("")

# A TensorFlow variable represents a reference to a tensor.
# The `read_value()` method provides access to the current value of the
# variable. Tensorflow Variables are automatically initialized according to the
# semantics defined in tf.get_variable().
print("Printing a TensorFlow Variable's value using .read_value():")
print(x.read_value())
print("")

print("Printing a TensorFlow Variable's value using .read_value().numpy():")
print(x.read_value().numpy())

Printing a TensorFlow Variable:
<tf.Variable 'x:0' shape=() dtype=float32, numpy=0.0>

Printing a TensorFlow Variable's value using .read_value():
tf.Tensor(0.0, shape=(), dtype=float32)

Printing a TensorFlow Variable's value using .read_value().numpy():
0.0


## Changing a TensorFlow Variable's value

To change a TensorFlow Variable's value, use its `.assign()` or `.assign_add()` method:

In [7]:
x.assign(42)
print(x.read_value())

x.assign_add(3)
print(x.read_value())

tf.Tensor(42.0, shape=(), dtype=float32)
tf.Tensor(45.0, shape=(), dtype=float32)


## Use a Variable just like any other Tensor

In [8]:
print(x + 3)

# This code will broadcast the value across the list of numbers:
print(x * [1, 2, 4])

Instructions for updating:
Colocations handled automatically by placer.
tf.Tensor(48.0, shape=(), dtype=float32)
tf.Tensor([ 45.  90. 180.], shape=(3,), dtype=float32)


# Step 5: Debug Errors with Instant Feedback

TensorFlow's eager execution helps you identify and debug runtime issues through interactive exploration of code snippets.

Below, we'll define a length-4 vector, and attempt two `tf.slice()` operations,
one being legal and the other being illegal, leading to a runtime error that is
raised immediately.

In [9]:
vector = tf.constant([10.0, 20.0, 30.0, 40.0])

In [10]:
# Works, because the values of `begin` and `size` (the 2nd and 3rd input
# arguments) are within the bound of `vector`.
print(tf.slice(vector, [1], [3]))

tf.Tensor([20. 30. 40.], shape=(3,), dtype=float32)


In [11]:
# The following does NOT work, because the value of `size` (the 3rd
# argument) causes the indices to go out of the bounds of `vector`. The
# error is raised immediately.
try:
  print(tf.slice(vector, [1], [4]))
except tf.OpError as e:
  print("Caught error: %s" % e)

Caught error: Expected size[0] in [0, 3], but got 4 [Op:Slice]


# Step 6: Timing a Matrix Multiply

You can place Tensors on the GPU by calling a Tensor's `.gpu()` method.

The first operation executing on the GPU may be slow as TensorFlow initializes. Subsequent uses will be much faster.

In [12]:
# Create some Tensors
SIZE = 1000
cpu_tensor = tf.random_normal([SIZE, SIZE])

In [13]:
# Time a CPU-based matrix multiplication

print("Time to conduct matmul on CPU:")
%time tf.matmul(cpu_tensor, cpu_tensor)

Time to conduct matmul on CPU:
CPU times: user 68.7 ms, sys: 30.7 ms, total: 99.3 ms
Wall time: 9.87 ms


<tf.Tensor: id=101, shape=(1000, 1000), dtype=float32, numpy=
array([[-12.259217 ,  13.066517 , -48.723373 , ...,   3.7362375,
         24.57964  ,  -0.521729 ],
       [-43.850727 ,   3.0061579, -22.340277 , ..., -59.40836  ,
          8.77694  ,  20.774178 ],
       [-60.35776  ,  -8.872137 ,  37.701164 , ...,  17.228958 ,
          3.7727156,   4.705341 ],
       ...,
       [-33.59124  ,   3.31077  ,  32.182358 , ..., -51.499672 ,
        -60.91821  ,   4.4043384],
       [  1.9427683,  23.274075 , -17.06929  , ...,  21.094751 ,
         15.016581 , -32.87684  ],
       [-13.990274 , -39.196095 , -49.32546  , ..., -26.750393 ,
          6.7022   ,  35.8215   ]], dtype=float32)>